In [ ]:
from google.colab import files
uploaded = files.upload()

Saving tweets.csv to tweets (1).csv


In [ ]:
import numpy
import pandas as pd
import nltk 
import nltk.corpus 
import gensim.models 
import gensim. downloader 
import sklearn.metrics 
import sklearn.ensemble 
import sklearn.naive_bayes 
import sklearn.model_selection 
import sklearn.feature_extraction.text

In [ ]:
# import io
df = pd.read_csv(io.BytesIO(uploaded['tweets.csv']))
# Dataset is now stored in a Pandas Dataframe

In [ ]:
df.head(5)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [ ]:
# Text preprocessing

def preprocessor (text):
  text = text.lower()
  tokenizer = nltk.RegexpTokenizer(r"\w+")
  words = tokenizer.tokenize(text)
  porter_stemmer = nltk.stem.porter.PorterStemmer()
  stemmed_words = [porter_stemmer.stem(w) for w in words]
  return " ". join(stemmed_words)


df["preprocessed_text"] = df["text"].apply(preprocessor)
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,preprocessed_text
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),virginamerica what dhepburn said
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),virginamerica plu you ve ad commerci to the ex...
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),virginamerica i didn t today must mean i need ...
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),virginamerica it s realli aggress to blast obn...
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),virginamerica and it s a realli big bad thing ...


In [ ]:
# Splitting the Data into Training and Test Sets

x = df["preprocessed_text"]
y = df["airline_sentiment"]
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y)

In [ ]:
# Feature Engineering 

vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(min_df=5) # consider those words which appreas five times. Defaulat vaule for N-Grams should be tunned
vectorizer.fit(x_train)


x_train = vectorizer.transform(x_train).toarray()
x_test = vectorizer.transform(x_test).toarray()
print("new x train:", x_train.shape)
print("new x test:", x_test.shape)

new x train: (10980, 2195)
new x test: (3660, 2195)


In [ ]:
# Train Model

parameters_grid = {
    "criterion": ["gini", "entropy"],
    "n_estimators": range (200, 310, 30)
    
}


model = sklearn.model_selection.GridSearchCV(sklearn.ensemble.RandomForestClassifier(),
                                            parameters_grid, scoring="accuracy", cv=5, n_jobs=-1)

model.fit(x_train, y_train)

print("Accuracy of best Random Forest classfier = {:.2f}".format (model.best_score_))
print("Best found hyperparameters of Random Forest classfier = {}".format (model.best_params_))

Accuracy of best Random Forest classfier = 0.76
Best found hyperparameters of Random Forest classfier = {'criterion': 'entropy', 'n_estimators': 290}


In [ ]:
# Test model
y_predicted = model.predict(x_test)
accuracy = sklearn.metrics.accuracy_score(y_test, y_predicted)
print("Accuracy = {:.2f}".format (accuracy))

Accuracy = 0.76


In [ ]:
# Predict the Sentiment of new Tweets

tweets = ["This is a very bad airline!", "I love it, good flights."]
encoded_tweets = vectorizer.transform(tweets).toarray()
predicted_class = model.predict(encoded_tweets)
predicted_class

array(['negative', 'positive'], dtype=object)

**An Althernatevie Feature Engineering **

In [ ]:
# Loading Word2Vec
word2vec_model = gensim.downloader.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:

x_word_embedding = []


stop_words = nltk.corpus.stopwords.words("english")
tokenizer = nltk.tokenize.TweetTokenizer()

for tweet in df["text"]:
    words = tokenizer.tokenize(tweet)
    feature_vector = numpy.zeros(300)
    ctr = 0
    
    for word in words:
        if word in word2vec_model and word not in stop_words:
            feature_vector += word2vec_model[word]
            ctr += 1
        
    if ctr > 0:
        feature_vector /= ctr
    x_word_embedding.append(feature_vector)



In [ ]:
# x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x_word_embedding, y)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(x_word_embedding, y)

In [ ]:
# Train Model

parameters_grid = {
    "criterion": ["gini", "entropy"],
    "n_estimators": range(200, 310, 30)
    
}
    
model = sklearn.model_selection.GridSearchCV(sklearn.ensemble.RandomForestClassifier(),
                                             parameters_grid, scoring="accuracy", cv=5, n_jobs=-1)
model.fit(x_train, y_train)
    
print("Accuracy of best Random Forest classfier = {:.2f}".format(model.best_score_))
print("Best found hyperparameters of Random Forest classfier = {}".format(model.best_params_))

Accuracy of best Random Forest classfier = 0.63
Best found hyperparameters of Random Forest classfier = {'criterion': 'entropy', 'n_estimators': 290}


In [ ]:
y_predicted = model.predict(x_test)
accuracy = sklearn.metrics.accuracy_score(y_test, y_predicted)
print("Accuracy = {:.2f}".format(accuracy))

Accuracy = 0.61
